In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.patches import Rectangle
%matplotlib inline

In [ ]:
sr13_sql = '''
select x.mgra, hs, increment, city, cpa
from [regional_forecast].[sr13_final].[mgrabase] x
join [regional_forecast].[sr13_final].[mgra13] y
on x.mgra = y.mgra
where scenario = 0'''
sr13_df = pd.read_sql(sr13_sql, mssql_engine)

In [ ]:
jurisdictions_names_sql = '''
    SELECT zone, name
      FROM data_cafe.ref.geography_zone
     WHERE geography_type_id = 150
'''
jur_name_df = pd.read_sql(jurisdictions_names_sql, mssql_engine)

In [ ]:
xref_geography_sql = '''
    SELECT mgra_13, luz_13, cocpa_13, cocpa_2016,
           jurisdiction_2016, cicpa_13
      FROM data_cafe.ref.vi_xref_geography_mgra_13'''
xref_geography_df = pd.read_sql(xref_geography_sql, mssql_engine)

In [ ]:
xref_geography_df['jur_or_cpa_id'] = xref_geography_df['cocpa_13']
xref_geography_df['jur_or_cpa_id'].fillna(xref_geography_df['cicpa_13'],inplace=True)
xref_geography_df['jur_or_cpa_id'].fillna(xref_geography_df['jurisdiction_2016'],inplace=True)
xref_geography_df['jur_or_cpa_id'] = xref_geography_df['jur_or_cpa_id'].astype(int)

In [ ]:
sr13_df = pd.merge(sr13_df,xref_geography_df[['mgra_13','jur_or_cpa_id','jurisdiction_2016']],\
                   left_on='mgra',right_on='mgra_13')

In [ ]:
sr13_df = pd.merge(sr13_df,jur_name_df[['zone','name']],left_on='city',right_on='zone')

In [ ]:
sr13_df.head()

In [ ]:
sr13_geo_df = pd.DataFrame({'hs_sum': sr13_df.groupby(['name','increment']).
                               hs.sum()}).reset_index()
sr13_geo_df.rename(columns = {'name':'geo'},inplace=True)
sr13_geo_df.head()

In [ ]:
sr13_geo_df_pivot = sr13_geo_df.pivot\
(index='increment', columns='geo', values='hs_sum').\
reset_index().rename_axis(None, axis=1)
sr13_geo_df_pivot.set_index('increment',inplace=True)
sr13_geo_df_pivot

In [ ]:
total_diff = sr13_geo_df_pivot.loc[[2012,2050],:]
differences = total_diff.diff().loc[[2050]]
differences.rename(index={2050: 'total_change'},inplace=True)
totalchange = pd.DataFrame(differences.sum(axis=0))
totalchange.rename(columns={0: 'total_change'},inplace=True)
print("\nTotal Units added sr13: {:,}".format(int(totalchange.total_change.sum())))

In [ ]:
totalchange

In [ ]:
sr13_geo_df_pivot

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=2)
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
plt.xticks(rotation=70)
plt.subplots_adjust(top=0.25)
plt.suptitle("\nTotal Additional Housing Units: {:,}".format(int(totalchange.total_change.sum())),size=18)
for j, jur in enumerate(sr13_geo_df.geo.unique().tolist()):
    df_plot = sr13_geo_df_pivot[[jur]]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,30),x_compat=True)
    axes.flat[j].set_xlabel("Forecast Year",size=12)
    axes.flat[j].set_ylabel("Total Housing Units",size=12)
    axes.flat[j].yaxis.set_major_formatter(tick) 
    axes.flat[j].set_xticks([2012,2015,2020,2025,2030,2035,2040,2045,2050])
    axes.flat[j].set_xticklabels([2012,2015,2020,2025,2030,2035,2040,2045,2050],rotation=45)
    axes.flat[j].set_title('SERIES 13 REGIONAL GROWTH FORECAST',size=14) 
    axes.flat[j].legend(fontsize=14)
    handles, labels = axes.flat[j].get_legend_handles_labels()
    extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    labels.append('chg = ' + str(int(totalchange.loc[jur][0])))
    handles.append(extra)
    axes.flat[j].legend(handles, labels)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.savefig('sr13.png', format='png', dpi=300)
